In [29]:
import bisect
import cv2
import itertools as it
import numpy as np
import pickle
import random
from collections import Counter, defaultdict
from utilities import *

In [20]:
file_titles = ('2020-04-11-telshin-3', '2020-04-13-mixwell-1', '2020-04-14-myth-0')
top, left, bottom, right = 39, 678, 242, 725
width, height = right - left, 11
color = [0.52949995, 0.6719118, 0.70135915]
color = [min(max(0, round(255 * v)), 255) for v in color]
color = np.array(color, dtype=np.uint8)
color_template = np.broadcast_to(color, (height, width, 3))
color

array([135, 171, 179], dtype=uint8)

In [15]:
def fn(file_title):
    input_file_path = fr'D:\Valorant\Videos\{file_title}.mp4'
    output_file_path = fr'D:\Valorant\Killed\Pickles\{file_title}.pickle'
    video = cv2.VideoCapture(input_file_path)
    with open(output_file_path, 'wb') as fout:
        while True:
            _, frame = video.read()
            if frame is None:
                break
            image = frame[top:bottom, left:right]
            pickle.dump(image, fout)
    video.release()
for s in file_titles:
    fn(s)

In [17]:
def fn(file_path):
    with open(file_path) as fin:
        return fin.readline().rstrip()
g = (fr'D:\Valorant\Killed\Labels\{s}.txt' for s in file_titles)
g = map(fn, g)
g = it.chain.from_iterable(g)
labels = ''.join(g)
Counter(labels)

Counter({'n': 84497, 'y': 30976, 'z': 1})

In [18]:
def fn(g, n, k):
    for v in g:
        if random.randrange(n) < k:
            yield v
            k -= 1
            if k == 0:
                break
        n -= 1
g = (fr'D:\Valorant\Killed\Pickles\{s}.pickle' for s in file_titles)
g = map(pickle_iter, g)
g = it.chain.from_iterable(g)
g = zip(g, labels)
g = ((i, l) for i, l in g if l != 'z')
data = list(fn(g, len(labels), 20000))
Counter(l for i, l in data)

Counter({'n': 14599, 'y': 5400})

In [23]:
def fn(image, label):
    match = cv2.matchTemplate(image, color_template, cv2.TM_SQDIFF)
    result = match.min()
    return label, result
d = defaultdict(list)
g = it.starmap(fn, data)
for label, result in g:
    d[label].append(result)

In [25]:
[(k, min(v), max(v)) for k, v in d.items()]

[('n', 28361.0, 39982580.0), ('y', 4158.0, 4745223.0)]

In [32]:
class Tesst:
    def __len__(self):
        return 4745225
    def __getitem__(self, i):
        def fn(image, label):
            match = cv2.matchTemplate(image, color_template, cv2.TM_SQDIFF)
            result = match.min() < i
            return label, result
        d = Counter(it.starmap(fn, data))
        n = d[('y', False)]
        print(i, d)
        return 13 - n
bisect.bisect_left(Tesst(), 0)

2372612 Counter({('n', False): 10341, ('y', True): 5326, ('n', True): 4258, ('y', False): 74})
3558919 Counter({('n', False): 9023, ('n', True): 5576, ('y', True): 5396, ('y', False): 4})
2965766 Counter({('n', False): 9633, ('y', True): 5366, ('n', True): 4966, ('y', False): 34})
3262343 Counter({('n', False): 9357, ('y', True): 5394, ('n', True): 5242, ('y', False): 6})
3114055 Counter({('n', False): 9502, ('y', True): 5391, ('n', True): 5097, ('y', False): 9})
3039911 Counter({('n', False): 9566, ('y', True): 5389, ('n', True): 5033, ('y', False): 11})
3002839 Counter({('n', False): 9599, ('y', True): 5387, ('n', True): 5000, ('y', False): 13})
2984303 Counter({('n', False): 9614, ('y', True): 5379, ('n', True): 4985, ('y', False): 21})
2993571 Counter({('n', False): 9605, ('y', True): 5387, ('n', True): 4994, ('y', False): 13})
2988937 Counter({('n', False): 9610, ('y', True): 5382, ('n', True): 4989, ('y', False): 18})
2991254 Counter({('n', False): 9607, ('y', True): 5387, ('n', 

2989721

In [33]:
def fn(image, label):
    match = cv2.matchTemplate(image, color_template, cv2.TM_SQDIFF)
    result = match.min() < 2990096
    return label, result
Counter(it.starmap(fn, data))

[Counter({('n', False): 9609,
          ('n', True): 4990,
          ('y', True): 5387,
          ('y', False): 13})]